# Running on ServiceX

Some short examples at this point.

If you disconnect from the internet in the middle of a query execution, it will continue to execute inside ServiceX. You can then re-run the query, and the software should realize it was interupted, and fetch the results from the query you already made.

## UpROOT

## ATLAS R21 XAOD

## CMS Run 1 AOD